# Time, Datetime, DateUtil

Author: Eni Mustafaraj  

This notebook contains mostly explanations and useful examples for the modules `time`, `datetime` and `dateutil`.
There are a few exercises for you to make sure that you understand the concepts.

**Table of Content**
1. [The `time` module](#sec1)
2. [The usages of `time`](#sec2)
3. [The `datetime` module](#sec3)  
.. Creating datetime objects  
.. From Unix epoch to datetime  
.. Comparing dates
4. [The `timedelta` class](#sec4)
5. [Arithmetic with `datetime` objects](#sec5)
6. [From `datetime` to string](#sec6)  
.. Exercise 1: String Conversion
7. [From string to `datetime`](#sec7)  
.. Exercise 2: Date Conversion
8. [The `dateutil` module](#sec8)
9. [Solutions](#sec9)

<a id="sec1"></a>

## 1. The `time` module
The built-in `time` module allows Python to read the system clock for the current time.

In [ ]:
import time
time.time()

The result that we get is the **number of seconds** that have elapsed since the Unix epoch, January 1st, 1970 at UTC.  This is an arbitrary day that was chosen as a fixed day, so that computer systems in different part of the world (with different timezones) can have it as a reference point.  

We can convert these seconds into years, and we'll see that they show around 52+ years (time between 1970 to today).

In [ ]:
oneYear = (60*60*24*365) # seconds * minutes * hours * days in a year
time.time()/oneYear      # convert into years

### Aside: Year 2038 problem

You might have already started hearing about the [Year 2038 problem](https://en.wikipedia.org/wiki/Year_2038_problem), or the "Unix Millenium Bug". This bug is the 32-bit overflow. What does that mean? You should take CS 240, it makes sense when you know about how data are stored in the computer memory. 

Briefly, many computers still use 32 bits to store information in memory. 1 bit is reserved, 31 bits are for numbers.  Given that the computer uses binary representation of numbers, we can represent numbers up to 2^31.

In [ ]:
2**31

Thus, this will be the largest number of seconds we can represent with 32 bits.

Now, let's see how will the epoch number look like in 2038. 

In [ ]:
totalSeconds = \
(68 *  # years from 1970 to 2038
365 *  # number of days in non-leap years
24  *  # number of hours per day
60  *  # number of minutes in a hour
60 + 
17*24*60*60) # assuming 17 leap years in this period: 1972, 1976, etc. to 2036

totalSeconds

Let's calucate now the number of days away from the Unix bug in 2039.

In [ ]:
print((2**31 - totalSeconds)/(24*60*60))

In fact, the expected "doom day" is January 19, 2038.

<a id="sec2"></a>

## 2. Usages of `time`

Two usages of `time` are to measure how long it takes to run a script, and to pause the execution for some seconds.

Let's create first a function tha might take some time to execute and then check how to use `time` to measure the execution period.

In [ ]:
def calcProd():
    """Calculate the product of the first 100000 numbers."""
    product = 1
    for i in range(1, 100000):
        product = product * i
    return product

In [ ]:
import sys
sys.set_int_max_str_digits(1000000) # it's here because some Mac versions limit the number of digits.

startTime = time.time()   # get the moment the execution starts
prod = calcProd()         # call the function
endTime = time.time()     # get the moment the execution ends

print(endTime - startTime, "seconds")
print("calculated product has", len(str(prod)), "digits.")

**Note:** Because the function `calcProd` is multiplying numbers that get larger and larger, this calculation will take a few seconds.

### `time.sleep()`

This function takes as an argument the number of seconds you want the program to **pause**.

In [ ]:
time.sleep(5)

We know that the program has paused, because of the star symbol showing in the `In [*]` label, which shows that the Jupyter kernel is busy.

In [ ]:
for i in range(3):
    print('Tick')
    time.sleep(1)
    print('Tock')
    time.sleep(1)

What else does this module have to offer? We can always use the function `dir` to learn more:

In [ ]:
print(dir(time))

Let's look at some of the properties and functions:

In [ ]:
time.timezone

The timezone is the number of seconds from the UTC (Coordinated Universal Time). Boston in the UTC-5:00 time zone, which corresponds to 18000 seconds.

In [ ]:
time.gmtime()

The `gmtime` refers to Greenwich Mean Time (GMT). The result is a datastructure that contains each part of the datetime in a separate attribute.

We can look at the details of this data structure by calling `help` on it:

In [ ]:
help(time.struct_time)

<a id="sec3"></a>

## 3. The `datetime` module

The `time` module has some good functions, but it's very hard to work with when it comes to date operations. For that purpose, we use the `datetime` module.

In [ ]:
import datetime
datetime.datetime.now()

What's happening in the above piece of code is the following:

1. `datetime` is a package (or module) that contains several classes
2. `datetime` is a class in the module `datetime`, and creates objects of the type `datetime`
3. `now()` is a method of the class `datetime` that tells the datetime at the current moment in time

In [ ]:
type(datetime)

In [ ]:
type(datetime.datetime)

In [ ]:
type(datetime.datetime.now)

The `datetime` module has several classes, which we can list by using the built-in function `dir`:

In [ ]:
dir(datetime)

### Creating `datetime` objects

We can call the constructor of the class `datetime` (it's the same as the class) with a list of arguments to create `datetime` objects.

**Example 1:** Create a new object by only providing the values for year, month, and date:

In [ ]:
datetime.datetime(2022, 9, 21)

**Example 2:** Create a new object by providing hour and minutes and seconds too:

In [ ]:
datetime.datetime(2022, 9, 21, 10, 15, 47)

Once we have a `datetime` object (either by invoking `now` or the datetime constructor), we can access all the part of the datetime as instance attributes:

In [ ]:
dt = datetime.datetime.now()
print("year:", dt.year)
print("month:", dt.month)
print("day:", dt.day)
print("hour:", dt.hour)
print("minute:", dt.minute)
print("second:", dt.second)

### From Unix epoch to datetime

The function `time.time()` returned the number seconds since the Unix epoch (Jan 1, 1970, 12AM, UTC). The Unix epoch time is often how time is stored in a system (because it's more efficient to store and the calculations with an integer are faster). Thus, `datetime.datetime` had a method to convert from Unix epoch to `datetime` object.

In [ ]:
datetime.datetime.fromtimestamp(10000) # 10000 seconds after Unix epoch

**Question:** What happened? Can you explain this result?  

**Answer:**

We can pass the function `time.time()` directly to the method `.fromtimestamp()` to craete the datetime object:

In [ ]:
datetime.datetime.fromtimestamp(time.time())

The last number you see is that of **microseconds**. There are 1 million microseconds in a second.

### Comparing dates
One of the best reasons for using `datetime` objects is that it makes the comparison of dates really easy (comparing string representation of dates is offen full of mistakes). Here are some examples:

In [ ]:
halloween = datetime.datetime(2022, 10, 31)
thanksgiving = datetime.datetime(2022, 11, 24)
newYear = datetime.datetime(2023, 1, 1)

In [ ]:
halloween > newYear

In [ ]:
thanksgiving > halloween

<a id="sec4"></a>

## 4. The `timedelta` class

The module `datetime`, in addition to the `datetime` class that creates moments in time, has also the class `timedelta`, which manipulates "time durations".  
In the following, it is shown how to:

- create a new `timedelta` instance by calling the class constructor
- perform arithmetic operations with datetime objects:
    - add or subtract an interval (timedelta instance) to an existing datetime instance
    - subtract two `datetime` objects to get as a result a `timedelta` object
    
We pass to the constructor named parameters for weeks, days, hours, minutes, seconds, milliseconds, and microseconds. 

In [ ]:
obj1 = datetime.timedelta(weeks=3, days=4)
obj1.days

In [ ]:
obj2 = datetime.timedelta(weeks=4, days=4, hours=4, minutes=4, seconds=24)
obj2

Once we have a `timedelta` object, we can only retrieve the number of seconds or that of the days from it:

In [ ]:
print(obj2.days, obj2.seconds)

But, we can also get the total number of seconds:

In [ ]:
obj2.total_seconds()

To make the output of a timedelta value readable, we pass it to the function `str`:

In [ ]:
str(obj2)

<a id="sec5"></a>

## 5. Arithmetic with `datetime` objects
If we have a `datetime.datetime` object, we can find another date in the past or the future by subtracting/adding a `timedelta` object.

In [ ]:
now = datetime.datetime.now()
thirtyDays = datetime.timedelta(days=30)

future = now + thirtyDays
print(future)

In [ ]:
past = now - thirtyDays
print(past)

The great thing about using such operations is that we don't have to care about leap years, different number of days in a month, etc.  All the arithmetic of dates is taken care by these two classes.

`timedelta` objects can also appear in multiplications and divisions:

In [ ]:
oneYear = datetime.timedelta(days=365)
twoYears = 2 * oneYear
print(twoYears)

In [ ]:
halfYear = oneYear/2
print(halfYear)

### Subtracting two datetime objects creates a timedelta object

In [ ]:
thanksgiving - halloween

In [ ]:
newYear - thanksgiving

**Note:** While it makes sense to subtract two dates to find the interval between them, there is no meaning in adding two datetime objects.  If you try it, you'll get an error.

In [ ]:
newYear + halloween

<a id="sec6"></a>

## 6. From `datetime` to string

The method `strftime` will be used to format a datetime object as a string, by making use of string formatting directives, a list of which with their meaning is shown in the table below:

| directive |                      meaning                      |
|:---------:|:-------------------------------------------------:|
| %Y        | Year with century, as in '2014'                   |
| %y        | Year without century, '00' to '99' (1970 to 2069) |
| %m        | Month as a decimal number, '01' to '12'           |
| %B        | Full month name, as in 'November'                 |
| %b        | Abbreviated month name, as in 'Nov'               |
| %d        | Day of the month, '01' to '31'                    |
| %j        | Day of the year, '001' to '366'                   |
| %w        | Day of the week, '0' (Sunday) to '6' (Saturday)   |
| %A        | Full weekday name, as in 'Monday'                 |
| %a        | Abbreviated weekday name, as in 'Mon'             |
| %H        | Hour (24-hour clock), '00' to '23'                |
| %I        | Hour (12-hour clock), '01' to '12'                |
| %M        | Minute, '00' to '59'                              |
| %S        | Second, '00' to '59'                              |
| %p        | 'AM' or 'PM'                                      |

Let's see some examples with different formatting strings:

In [ ]:
now = datetime.datetime.now()
now

In [ ]:
print(now.strftime("It's %B '%y"))

**Explanation:** You provide a template string with some slots (or holes), which are represented with the directives that consists of the `%` character and one letter from the table above. The instance variable values for the object `now` are inserted in the "holes" of the templates.  
Each of the directives specifies which instance variable goes where.

In [ ]:
print(now.strftime("Today is %A, %Y/%m/%d."))

In [ ]:
print(now.strftime('The time is %I:%M %p'))

In [ ]:
print(now.strftime('%Y/%m/%d %H:%M:%S'))

### Exercise 1: String conversion

Write the expression that use `datetime` string directives that will create the following text:

   `Today is Thu, the 14th of Oct '21 and the 287th day of the year 2021.`
   
Your day doesn't need to be Oct 14, it can be whatever the current day is. Thus, the number of the day of the year will also be different from 287.

**Hint:** Use the table of directives above to figure out what directives to include.

In [ ]:
# write your code here


<a id="sec7"></a>

## 7. From string to `datetime`

Very often we will encounter the opposite problem: read a text value from a file which needs to be represented internally as a datetime object.  
This process is known as parsing and is performed by the method `.strptime()`. This method takes two parameters:

1. the string to be parsed
2. the custom format string (that indicates the meaning of the components in the string)

For example, the string `'October 14, 2021'`, is presented by the format string `'%B %d, %Y'`.

In [ ]:
datetime.datetime.strptime('October 14, 2021', '%B %d, %Y')

In [ ]:
datetime.datetime.strptime('2019/10/14 7:34 PM', '%Y/%m/%d %I:%M %p')

### Be careful with some conversions

Let's try to parse this date that shows up in emails opened within Gmail (if you click the option "Show original"):

```
Wed, 13 Oct 2021 14:56:46 -0700 (PDT)
```

This date contains **timezone** information, and according to the docs, there are two directives to handle them: `'%z'` and `'%Z'`.

**ERROR:** However, it turns out that %z and %Z are platform-specific (that is, they don't work on all computers).

This is what the Python documentation says:

    The full set of format codes supported varies across platforms, because Python calls the platform C library’s strftime() function, and platform variations are common. 

In [ ]:
date = "Wed, 13 Oct 2021 14:56:46 -0070 (PDT)"
datetime.datetime.strptime(date, "%a, %d %b %Y %H:%M:%S %z (%Z)")

**Note:** We'll overcome this error by using the module `dateutil`, see next Section.

### Exercise 2: Date conversion

Here is how timestamps show up in Wikipedia edit lists:

    08:59, 20 September 2022
    
Write a statement to convert this string into a datetime object.

In [ ]:
# your code here


### `strftime` versus `strptime`

We discussed two methods in the past two sections. These methods have very similar names, but different signatures.

1. `now.strftime('%Y/%m/%d %H:%M:%S')` => Format a date object as a string, given the string format
2. `datetime.datetime.strptime('October 14, 2021', '%B %d, %Y')` => Parse a string into a date object

The letters **f** and **p** in the names of the function stand for the words: format and parse.


<a id="sec8"></a>

## 8. The `dateutil` module

To deal with the issue of the timezone implementation, we'll use a new library.  

* try `import dateutil`. If you get an error install it by: `pip install dateutil` inside the notebook.

In [ ]:
import dateutil

In [ ]:
from dateutil import parser
date = "Wed, 13 Oct 2021 14:56:46 -0070 (PDT)"
parser.parse(date)

Here is another example of a date, that is the one used by the `mbox` Python module that allows us to work with an inbox of emails (if we download our inbox).

In [ ]:
date = "Mon Oct 11 11:11:06 +0000 2021"
parser.parse(date)

**Note:** As you can see the value of the `dateutil` parser is to make away with having to write string directives for the conversion. But, it doesn't always work, thus, it's good to know about `datetime` too.

Knowing about `datetime` is also important because `dateutil` creates a `datetime` object, and all arithmetic with dates is via `datetime` methods.

<a id="sec9"></a>
## 9. Solutions

**Section 3 - Question on Unix Epochs**  
How to explain `datetime.datetime(1969, 12, 31, 21, 46, 40)`?  
When it was Jan 1st, 00:00 in GMT (London), our local time was still Dec 31st 1969. This is why 10000 seconds into the 1970 year shows what was the datetime for us in Boston.


**Section 6 - Exercise 1**
```
now.strftime("Today is %a, the %dth of %b '%y and the %jth day of the year %Y")
```

**Section 7 - Exercise 2**
```
date = "08:59, 20 September 2022"
datetime.datetime.strptime(date, "%H:%M, %d %B %Y")
```